In [40]:
import numpy as np

In [41]:
good_nodes = np.array([0, 1, 2, 3])
bad_nodes = np.array([4, 5, 6])

### Adjacency Matrix

In [42]:
adj_matrix = np.zeros((7, 7))
adj_matrix[0, 1] = 1
adj_matrix[1, 2] = 1
adj_matrix[1, 3] = 1
adj_matrix[2, 1] = 1
adj_matrix[3, 4] = 1
adj_matrix[4, 5] = 1
adj_matrix[4, 6] = 1
adj_matrix[5, 2] = 1

In [43]:
# adj_matrix = np.array([
#                        [0, 1, 0, 0],
#                        [0, 0, 1, 0],
#                        [0, 1, 0, 1],
#                        [0, 0, 0, 0]
# ])

### Build Transition and Inverse Transition Matrix

In [44]:
def makeT(adj_matrix):
  T = np.zeros(adj_matrix.shape)

  for p in range(T.shape[0]):
    for q in range(T.shape[1]):
      if adj_matrix[q, p] == 1:
        T[p, q] = 1 / adj_matrix[q, :].sum()

  return T

T = makeT(adj_matrix)
print(T)

[[0.  0.  0.  0.  0.  0.  0. ]
 [1.  0.  1.  0.  0.  0.  0. ]
 [0.  0.5 0.  0.  0.  1.  0. ]
 [0.  0.5 0.  0.  0.  0.  0. ]
 [0.  0.  0.  1.  0.  0.  0. ]
 [0.  0.  0.  0.  0.5 0.  0. ]
 [0.  0.  0.  0.  0.5 0.  0. ]]


In [45]:
def makeInvT(adj_matrix):
  T = np.zeros(adj_matrix.shape)

  for p in range(T.shape[0]):
    for q in range(T.shape[1]):
      if adj_matrix[p, q] == 1:
        if adj_matrix[:, q].sum() != 0:
          T[p, q] = 1 / adj_matrix[:, q].sum()
        else:
          T[p, q] = 0

  return T

U = makeInvT(adj_matrix)
print(U)

[[0.  0.5 0.  0.  0.  0.  0. ]
 [0.  0.  0.5 1.  0.  0.  0. ]
 [0.  0.5 0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  1.  0.  0. ]
 [0.  0.  0.  0.  0.  1.  1. ]
 [0.  0.  0.5 0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0. ]]


### Select Seed

In [46]:
def SelectSeed(U, N, alpha, M):
  s = np.ones(N)
  ones = np.ones(N)
  for i in range(M):
    s = alpha * (U @ s) + ((1 - alpha) / N) * ones

  return s

s = SelectSeed(U, 7, 0.85, 20)
print(s)

[0.08016994 0.13790991 0.08016994 0.09571493 0.0873091  0.05580169
 0.02142857]


### Rank

In [47]:
def Rank(x, s):
  combined = [i for _, i in sorted(zip(s, x))]
  combined = np.flipud(combined)
  return combined

sigma = Rank(np.arange(7), s)
print(sigma)

[1 3 4 2 0 5 6]


# Trust Rank Algorithm

In [53]:
def TrustRank(adj_matrix, N, L, alpha_B, M_B, alpha_I, M_I, good_nodes, bad_nodes):
  T = makeT(adj_matrix)
  U = makeInvT(adj_matrix)
  s = SelectSeed(U, N, alpha_I, M_I)

  sigma = Rank(np.arange(N), s)

  d = np.zeros(N)
  for i in range(L):
    if sigma[i] in good_nodes:
      d[sigma[i]] = 1

  d = d / d.sum()

  t_star = d.copy()
  for i in range(M_B):
    t_star = alpha_B * (T @ t_star) + (1 - alpha_B) * d

  return t_star

t_star = TrustRank(adj_matrix, 7, 3, 0.85, 20, 0.85, 20, good_nodes, bad_nodes)
results = np.round(t_star, decimals = 2)
print("Results:", results)

Results: [0.   0.18 0.12 0.15 0.13 0.05 0.05]
